# Lecture 2: Introduction to Feature Engineering

## Setup

In [1]:
import numpy as np 
import pandas as pd
import random
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn import preprocessing
import zipfile
import io
import zipfile



In [3]:
articles = pd.read_csv('../../articles.csv')
customers = pd.read_csv('../../customers.csv')
# sample_submisison = pd.read_csv('sample_submission.csv')
transactions = pd.read_csv('../../transactions_train.csv')

## The H&M Dataset

In [584]:
# articles.info

In [585]:
# customers.info()

In [586]:
# transactions.info()

In [587]:
# X = transactions.merge(customers, how='inner', on='customer_id')
# X = X.merge(articles, how='inner', on='article_id')

### Creating Samples 
If you would rather work with samples instead of the whole dataset (while prototyping your code). You can use the code below:

In [4]:
# Adapted from: https://www.kaggle.com/code/paweljankiewicz/hm-create-dataset-samples
# This extracts three sampled datasets, containing 0.1%, 1% and 5% of all users and their transactions, and the associated articles.
for sample_repr, sample in [("01", 0.001), ("1", 0.01), ("5", 0.05)]:
    print(sample)
    customers_sample = customers.sample(int(customers.shape[0]*sample), replace=False)
    customers_sample_ids = set(customers_sample["customer_id"])
    transactions_sample = transactions[transactions["customer_id"].isin(customers_sample_ids)]
    articles_sample_ids = set(transactions_sample["article_id"])
    articles_sample = articles[articles["article_id"].isin(articles_sample_ids)]
    customers_sample.to_csv(f"customers_sample{sample_repr}.csv.gz", index=False)
    transactions_sample.to_csv(f"transactions_sample{sample_repr}.csv.gz", index=False)
    articles_sample.to_csv(f"articles_sample{sample_repr}.csv.gz", index=False)

0.001
0.01
0.05


In [589]:
# articles = pd.read_csv('articles_sample01.csv.gz')
# customers = pd.read_csv('customers_sample01.csv.gz')
# transactions = pd.read_csv('transactions_sample01.csv.gz')

In [590]:
# customers_sample.info()

In [591]:
# transactions_sample.info()

## A Simplified Task: Binary Classification

The task of predicting which 12 items users are most likely to buy in the next week is difficult to translate to a traditional classification machine learning setting. 
To obtain the 12 items a user is most likely to buy, we need to make predictions for all items (or the ones selected by a baseline) and select the 12 that have the highest predicted scores.

In this assignment, we'll consider a simplified task: Predict whether a user ordered a single item or not, based on the features of the user and the item. 
We provide a baseline logistic regression model below, but haven't done much feature preprocessing or engineering!
Initially, it is always best to focus your efforts on getting your features in the right shape and setting up the right validation scheme and baselines.
Once you are sure that your features add value and your validation scheme is correct, then you typically move on to trying more elaborate models.

### Creating the Dataset

In [592]:
# If you'd rather use a sample. Uncomment the following code:
# transactions = transactions_sample
# customers = customers_sample
# articles = articles_sample

In [593]:
transactions['ordered'] = 1

The problem setting is an example of a "PU learning" problem, i.e. only positives are labeled, everything else is unlabeled (and can be either positive or negative). 
Of course, we cannot train a classifier with just positive samples: The classifier will just learn that everything is positive.
Therefore, we need to manually generate negative samples.

Below, we use a simple random negative sampling strategy.
We want to create a balanced dataset, meaning that we have just as many positives as negatives.
This makes sure that the classifier will not benefit from predicting the positive/negative class more often than the other.
Realistically, the amount of positive samples is of course many times smaller than the amount of unlabeled, possibly negative instances.


If you want to try your hand at a more complex negative sampling strategy, you may want to check out this blog as a starting point: https://medium.com/mlearning-ai/overview-negative-sampling-on-recommendation-systems-230a051c6cd7.



In [594]:
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,ordered
0,2018-09-20,017ddc94fee40ab9a24af073a4c69282b4c04e094d53a9...,496111020,0.033881,1,1
1,2018-09-20,01f0287961a0f7453c24e6aede462f55f667e705156f1e...,635425004,0.016932,2,1
2,2018-09-20,01f0287961a0f7453c24e6aede462f55f667e705156f1e...,505071001,0.023712,2,1
3,2018-09-20,082fa45b5ed4935d58b92bc9044e5a249772a740d9d2cc...,587115001,0.011847,2,1
4,2018-09-20,74303ec6a09511c0d565953f448f64a07f769033df8f0a...,628633001,0.025407,2,1


In [595]:
# What's happening here? 
# We're creating negative samples. I.e. we're creating transactions that didn't actually occur.
# First, we need to know which interactions did occur:
positive_pairs = list(map(tuple, transactions[['customer_id', 'article_id']].drop_duplicates().values))

In [596]:
# Then we need to know what every synthetic transaction should contain: a date, a customer_id, an article_id, price, sales_channel_id. We will set ordered = 0, as these transactions didn't really occur.
transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,ordered
0,2018-09-20,017ddc94fee40ab9a24af073a4c69282b4c04e094d53a9...,496111020,0.033881,1,1
1,2018-09-20,01f0287961a0f7453c24e6aede462f55f667e705156f1e...,635425004,0.016932,2,1
2,2018-09-20,01f0287961a0f7453c24e6aede462f55f667e705156f1e...,505071001,0.023712,2,1
3,2018-09-20,082fa45b5ed4935d58b92bc9044e5a249772a740d9d2cc...,587115001,0.011847,2,1
4,2018-09-20,74303ec6a09511c0d565953f448f64a07f769033df8f0a...,628633001,0.025407,2,1


In [597]:
# Extract real values
real_dates = transactions["t_dat"].unique()
real_customers = transactions["customer_id"].unique()
real_articles = transactions["article_id"].unique()
real_channels = transactions["sales_channel_id"].unique()
article_and_price = transactions[["article_id","price"]].drop_duplicates("article_id").set_index("article_id").squeeze()

In [598]:
# How many negatives do we need to sample?
num_neg_pos = transactions.shape[0]
print(num_neg_pos)

31816


In [599]:
# Sampling negatives by selecting random users, articles, dates and sales channel:
# Note: This is quite naive. Some articles may not even have been available at the date we are sampling.
random.seed(42)

# Afterwards, we need to remove potential duplicates, so we'll sample too many.
num_neg_samples = int(num_neg_pos * 1.1)

# Sample each of the independent attributes.
neg_dates = np.random.choice(real_dates, size=num_neg_samples)
neg_articles = np.random.choice(real_articles, size=num_neg_samples)
neg_customers = np.random.choice(real_customers, size=num_neg_samples)
neg_channels = np.random.choice(real_channels, size=num_neg_samples)
ordered = np.array([0] * num_neg_samples)
# Assign to every article a real price.
neg_prices = article_and_price[neg_articles].values

In [600]:
neg_transactions = pd.DataFrame([neg_dates, neg_customers, neg_articles, neg_prices, neg_channels, ordered], index=transactions.columns).T

In [601]:
# Result:
neg_transactions.head()

,t_dat,customer_id,article_id,price,sales_channel_id,ordered
0,2019-08-24,9883dc9ec2b54012d4c249042bc0baa4edb50059cbb109...,189616007,0.016932,2,0
1,2020-07-25,9806695c05b9c8cc4f096f2b27ab93a4c33efcf8f01f55...,728997001,0.025407,1,0
2,2020-05-19,7527e040459b3fcb6cfc6c0ed0c17b2ee379175add9d22...,757906005,0.025407,2,0
3,2019-08-10,8e1179700c6224702bfb5550214b45a23d810ad1baf200...,495685014,0.040661,2,0
4,2019-09-18,7565e93c32086c9d8e265ccb9691cc78bbaf546c9494da...,858052007,0.025407,2,0


In [602]:
neg_transactions.shape

(34997, 6)

In [603]:
# Remove random negative samples that actually coincide with positives
df = neg_transactions[
    ~neg_transactions.set_index(["customer_id", "article_id"]).index.isin(positive_pairs)
]

# Remove any excess
chosen_neg_transactions = df.sample(num_neg_pos)

In [604]:
# Concat the negative samples to the positive samples:
transactions = pd.concat([transactions, chosen_neg_transactions])
transactions = transactions.merge(customers, how="inner", on='customer_id')
transactions = transactions.merge(articles, how="inner", on='article_id')

In [605]:
transactions.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 63632 entries, 0 to 63631
Data columns (total 36 columns):
 #   Column                        Non-Null Count  Dtype  
---  ------                        --------------  -----  
 0   t_dat                         63632 non-null  object 
 1   customer_id                   63632 non-null  object 
 2   article_id                    63632 non-null  object 
 3   price                         63632 non-null  object 
 4   sales_channel_id              63632 non-null  object 
 5   ordered                       63632 non-null  object 
 6   FN                            24653 non-null  float64
 7   Active                        24295 non-null  float64
 8   club_member_status            63485 non-null  object 
 9   fashion_news_frequency        63228 non-null  object 
 10  age                           63173 non-null  float64
 11  postal_code                   63632 non-null  object 
 12  product_code                  63632 non-null  int64  
 13  p

### Basic Preprocessing
Some very basic preprocessing.

In [606]:
# I'm dropping a lot of columns, use them in your engineering tasks!
transactions_processed = transactions[['customer_id', 'age', 'article_id', 'sales_channel_id', 'price', 'ordered']].copy()
transactions_processed.head()

,customer_id,age,article_id,sales_channel_id,price,ordered
0,017ddc94fee40ab9a24af073a4c69282b4c04e094d53a9...,28.0,496111020,1,0.033881,1
1,3f5c5f836c9b82e18a84c37b10a986a51f8a6a3d250c10...,40.0,496111020,1,0.027102,1
2,2b3ac8aafad8738d959cfbcc9c53e94d5de06345354965...,28.0,496111020,2,0.022017,1
3,2b3ac8aafad8738d959cfbcc9c53e94d5de06345354965...,28.0,496111020,2,0.022017,1
4,017ddc94fee40ab9a24af073a4c69282b4c04e094d53a9...,28.0,663007003,1,0.042356,1


In [607]:
# Does it make sense to label encode?
# Label encoding the customer and article IDs:
customer_encoder = preprocessing.LabelEncoder()
article_encoder = preprocessing.LabelEncoder()

In [608]:
transactions_processed['customer_id'] = customer_encoder.fit_transform(transactions_processed['customer_id'])
transactions_processed['article_id'] = article_encoder.fit_transform(transactions_processed['article_id'])

In [609]:
# If you want to go back to the original encoding:
customer_encoder.inverse_transform([2])

array(['0054357b9f186ddab5b7aa49d912929fb9895d282232625eda6dd53632a54cfd'],
      dtype=object)

In [610]:
transactions_processed.head()

,customer_id,age,article_id,sales_channel_id,price,ordered
0,8,28.0,866,1,0.033881,1
1,329,40.0,866,1,0.027102,1
2,223,28.0,866,2,0.022017,1
3,223,28.0,866,2,0.022017,1
4,8,28.0,4933,1,0.042356,1


In [611]:
# Can you come up with a NaN strategy that makes sense for each column in the dataset?
# Imputing all NaN values with zeros:
transactions_processed = transactions_processed.fillna(0)
transactions_processed.isnull().values.any()

False

In [612]:
# Does it make sense to one-hot encode?
# One-hot-encoding sales_channel_id:
transactions_processed = pd.get_dummies(transactions_processed, columns=['sales_channel_id'])

In [613]:
transactions_processed.head()

,customer_id,age,article_id,price,ordered,sales_channel_id_1,sales_channel_id_2
0,8,28.0,866,0.033881,1,1,0
1,329,40.0,866,0.027102,1,1,0
2,223,28.0,866,0.022017,1,0,1
3,223,28.0,866,0.022017,1,0,1
4,8,28.0,4933,0.042356,1,1,0


In [614]:
# Creating a Train / Test Split:
X_train, X_test, y_train, y_test = train_test_split(transactions_processed.drop('ordered', axis=1), transactions_processed['ordered'], test_size=0.10, random_state=42)

In [615]:
X_train.head()

,customer_id,age,article_id,price,sales_channel_id_1,sales_channel_id_2
45732,1154,33.0,8117,0.033881,0,1
31387,709,50.0,442,0.016932,0,1
18783,466,23.0,34,0.011847,1,0
16765,3,62.0,1132,0.016932,0,1
47263,158,27.0,16172,0.016932,0,1


In [616]:
y_train.head()

45732    1
31387    1
18783    1
16765    0
47263    1
Name: ordered, dtype: int64

## Baseline Model

In [617]:
# Will take a few minutes to run, if you're using the whole dataset:
# baseline = LogisticRegression(random_state=42)
# baseline = baseline.fit(X_train, y_train)

In [618]:
# baseline.predict_proba(X_test)

In [619]:
# y_test

In [620]:
# Mean Accuracy:
# baseline.score(X_test, y_test)
# As you can seen, the accuracy is ~0.51. In other words, the classifier predicts correctly 51% of the time whether a customer did or din't buy an item.
# Can you improve this baseline logistic regression model by doing better preprocessing and generating new features?
# Also think about my steps! Did it make sense to include the article and customer ids? (And things like that)

In [621]:
# Classification Metrics:
# predictions = baseline.predict(X_test)
# print(classification_report(y_test, predictions))

In [622]:
# predictions

## Assignment: Feature engineering
**TODO:** 
- In groups (of 2-3 students), think about (a few) features that can be engineered (preprocess and generate new features). Divide the work!
- Do these engineered features improve the baseline model?
- Add your thoughts & results to a slide deck for discussion next week (again, 1 slide per person).

# Material-product type #
Looks at the interaction between product type and the color of the item


In [623]:
transactions_processed = transactions[['customer_id', 'age', 'article_id', 'sales_channel_id', 'price', 'ordered', 'colour_group_code', 'colour_group_name', 'product_type_name']]
transactions_processed.head()

,customer_id,age,article_id,sales_channel_id,price,ordered,colour_group_code,colour_group_name,product_type_name
0,017ddc94fee40ab9a24af073a4c69282b4c04e094d53a9...,28.0,496111020,1,0.033881,1,43,Dark Red,Sweater
1,3f5c5f836c9b82e18a84c37b10a986a51f8a6a3d250c10...,40.0,496111020,1,0.027102,1,43,Dark Red,Sweater
2,2b3ac8aafad8738d959cfbcc9c53e94d5de06345354965...,28.0,496111020,2,0.022017,1,43,Dark Red,Sweater
3,2b3ac8aafad8738d959cfbcc9c53e94d5de06345354965...,28.0,496111020,2,0.022017,1,43,Dark Red,Sweater
4,017ddc94fee40ab9a24af073a4c69282b4c04e094d53a9...,28.0,663007003,1,0.042356,1,9,Black,Sweater


In [624]:
transactions_processed["colour_type"] = (transactions_processed.colour_group_name.astype(str) + '_' + transactions_processed.product_type_name.astype(str))
transactions_processed = transactions_processed.drop(['colour_group_name','product_type_name'], axis=1)
colour_type_encoder = preprocessing.LabelEncoder()
transactions_processed['colour_type'] = customer_encoder.fit_transform(transactions_processed['colour_type'])

transactions_processed.head()

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


,customer_id,age,article_id,sales_channel_id,price,ordered,colour_group_code,colour_type
0,017ddc94fee40ab9a24af073a4c69282b4c04e094d53a9...,28.0,496111020,1,0.033881,1,43,458
1,3f5c5f836c9b82e18a84c37b10a986a51f8a6a3d250c10...,40.0,496111020,1,0.027102,1,43,458
2,2b3ac8aafad8738d959cfbcc9c53e94d5de06345354965...,28.0,496111020,2,0.022017,1,43,458
3,2b3ac8aafad8738d959cfbcc9c53e94d5de06345354965...,28.0,496111020,2,0.022017,1,43,458
4,017ddc94fee40ab9a24af073a4c69282b4c04e094d53a9...,28.0,663007003,1,0.042356,1,9,118


In [625]:



customer_encoder = preprocessing.LabelEncoder()
article_encoder = preprocessing.LabelEncoder()
transactions_processed['customer_id'] = customer_encoder.fit_transform(transactions_processed['customer_id'])
transactions_processed['article_id'] = article_encoder.fit_transform(transactions_processed['article_id'])

transactions_processed = transactions_processed.fillna(0)
transactions_processed.isnull().values.any()
transactions_processed.head()

,customer_id,age,article_id,sales_channel_id,price,ordered,colour_group_code,colour_type
0,8,28.0,866,1,0.033881,1,43,458
1,329,40.0,866,1,0.027102,1,43,458
2,223,28.0,866,2,0.022017,1,43,458
3,223,28.0,866,2,0.022017,1,43,458
4,8,28.0,4933,1,0.042356,1,9,118


In [626]:
transactions_processed = pd.get_dummies(transactions_processed, columns=['sales_channel_id', 'colour_type'])
transactions_processed.head()

,customer_id,age,article_id,price,ordered,colour_group_code,sales_channel_id_1,sales_channel_id_2,colour_type_0,colour_type_1,...,colour_type_1391,colour_type_1392,colour_type_1393,colour_type_1394,colour_type_1395,colour_type_1396,colour_type_1397,colour_type_1398,colour_type_1399,colour_type_1400
0,8,28.0,866,0.033881,1,43,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,329,40.0,866,0.027102,1,43,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,223,28.0,866,0.022017,1,43,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,223,28.0,866,0.022017,1,43,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
4,8,28.0,4933,0.042356,1,9,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [627]:
X_train, X_test, y_train, y_test = train_test_split(transactions_processed.drop('ordered', axis=1), transactions_processed['ordered'], test_size=0.10, random_state=42)
baseline = LogisticRegression(random_state=42)
baseline = baseline.fit(X_train, y_train)
baseline.score(X_test, y_test)


0.6046511627906976

#Populartity#
Looks at what items are sold within the last 4 months

In [628]:
dated_transactions = transactions.loc[transactions["ordered"] == 1][["article_id", "t_dat"]]
dated_transactions["t_dat"] = pd.to_datetime(dated_transactions["t_dat"])
latest_date = dated_transactions["t_dat"].max()
dated_transactions["transaction_age_months"] = 12 * (latest_date.year - dated_transactions["t_dat"].dt.year) + (latest_date.month - dated_transactions["t_dat"].dt.month)
dated_transactions.head()

,article_id,t_dat,transaction_age_months
0,496111020,2018-09-20,24
1,496111020,2018-09-29,24
2,496111020,2019-01-03,20
3,496111020,2019-01-03,20
4,663007003,2018-09-24,24


Filter out sales that are older than 4 months. 

In [629]:
popular_items = dated_transactions.loc[dated_transactions["transaction_age_months"] < 4]
popular_items = popular_items["article_id"].value_counts().rename_axis("article_id").reset_index(name="count")

In [630]:
transactions_processed = transactions[['customer_id', 'age', 'article_id', 'sales_channel_id', 'price', 'ordered']].copy()
transactions_processed = transactions_processed.merge(popular_items, how="inner", on='article_id')
transactions_processed.head()

,customer_id,age,article_id,sales_channel_id,price,ordered,count
0,017ddc94fee40ab9a24af073a4c69282b4c04e094d53a9...,28.0,179123001,1,0.016932,1,1
1,017ddc94fee40ab9a24af073a4c69282b4c04e094d53a9...,28.0,179123001,1,0.016932,1,1
2,017ddc94fee40ab9a24af073a4c69282b4c04e094d53a9...,28.0,179123001,1,0.010153,1,1
3,74303ec6a09511c0d565953f448f64a07f769033df8f0a...,33.0,179123001,2,0.016932,1,1
4,1c58bc8c44ee0e1c203c7cadd1cbd234ead302850ae5a9...,38.0,179123001,1,0.01439,1,1


In [631]:
customer_encoder = preprocessing.LabelEncoder()
article_encoder = preprocessing.LabelEncoder()
transactions_processed['customer_id'] = customer_encoder.fit_transform(transactions_processed['customer_id'])
transactions_processed['article_id'] = article_encoder.fit_transform(transactions_processed['article_id'])
transactions_processed = transactions_processed.fillna(0)
transactions_processed.isnull().values.any()
transactions_processed.head()

,customer_id,age,article_id,sales_channel_id,price,ordered,count
0,8,28.0,7,1,0.016932,1,1
1,8,28.0,7,1,0.016932,1,1
2,8,28.0,7,1,0.010153,1,1
3,595,33.0,7,2,0.016932,1,1
4,145,38.0,7,1,0.014390,1,1


In [632]:
transactions_processed = pd.get_dummies(transactions_processed, columns=['sales_channel_id'])
transactions_processed.head()

,customer_id,age,article_id,price,ordered,count,sales_channel_id_1,sales_channel_id_2
0,8,28.0,7,0.016932,1,1,1,0
1,8,28.0,7,0.016932,1,1,1,0
2,8,28.0,7,0.010153,1,1,1,0
3,595,33.0,7,0.016932,1,1,0,1
4,145,38.0,7,0.014390,1,1,1,0


In [633]:
X_train, X_test, y_train, y_test = train_test_split(transactions_processed.drop('ordered', axis=1), transactions_processed['ordered'], test_size=0.10, random_state=42)
baseline = LogisticRegression(random_state=42)
baseline = baseline.fit(X_train, y_train)
baseline.score(X_test, y_test)

0.6298449612403101

#Popular product_type by colour
Looks at all items that were sold within the last 4 months and then
Looks at the interaction between product type and the color of the item

In [634]:
transactions_processed = transactions[['customer_id', 'age', 'article_id', 'sales_channel_id', 'price', 'ordered', 'colour_group_code', 'colour_group_name', 'product_type_name']]
transactions_processed.head()

,customer_id,age,article_id,sales_channel_id,price,ordered,colour_group_code,colour_group_name,product_type_name
0,017ddc94fee40ab9a24af073a4c69282b4c04e094d53a9...,28.0,496111020,1,0.033881,1,43,Dark Red,Sweater
1,3f5c5f836c9b82e18a84c37b10a986a51f8a6a3d250c10...,40.0,496111020,1,0.027102,1,43,Dark Red,Sweater
2,2b3ac8aafad8738d959cfbcc9c53e94d5de06345354965...,28.0,496111020,2,0.022017,1,43,Dark Red,Sweater
3,2b3ac8aafad8738d959cfbcc9c53e94d5de06345354965...,28.0,496111020,2,0.022017,1,43,Dark Red,Sweater
4,017ddc94fee40ab9a24af073a4c69282b4c04e094d53a9...,28.0,663007003,1,0.042356,1,9,Black,Sweater


In [635]:
transactions_processed = transactions_processed.merge(popular_items, how="inner", on='article_id')
transactions_processed["colour_type"] = (transactions_processed.colour_group_name.astype(str) + '_' + transactions_processed.product_type_name.astype(str))
transactions_processed = transactions_processed.drop(['colour_group_name','product_type_name'], axis=1)
colour_type_encoder = preprocessing.LabelEncoder()
transactions_processed['colour_type'] = customer_encoder.fit_transform(transactions_processed['colour_type'])
transactions_processed.head()

,customer_id,age,article_id,sales_channel_id,price,ordered,colour_group_code,count,colour_type
0,017ddc94fee40ab9a24af073a4c69282b4c04e094d53a9...,28.0,179123001,1,0.016932,1,9,1,65
1,017ddc94fee40ab9a24af073a4c69282b4c04e094d53a9...,28.0,179123001,1,0.016932,1,9,1,65
2,017ddc94fee40ab9a24af073a4c69282b4c04e094d53a9...,28.0,179123001,1,0.010153,1,9,1,65
3,74303ec6a09511c0d565953f448f64a07f769033df8f0a...,33.0,179123001,2,0.016932,1,9,1,65
4,1c58bc8c44ee0e1c203c7cadd1cbd234ead302850ae5a9...,38.0,179123001,1,0.01439,1,9,1,65


In [636]:
customer_encoder = preprocessing.LabelEncoder()
article_encoder = preprocessing.LabelEncoder()
transactions_processed['customer_id'] = customer_encoder.fit_transform(transactions_processed['customer_id'])
transactions_processed['article_id'] = article_encoder.fit_transform(transactions_processed['article_id'])
transactions_processed = transactions_processed.fillna(0)
transactions_processed.isnull().values.any()
transactions_processed.head()

,customer_id,age,article_id,sales_channel_id,price,ordered,colour_group_code,count,colour_type
0,8,28.0,7,1,0.016932,1,9,1,65
1,8,28.0,7,1,0.016932,1,9,1,65
2,8,28.0,7,1,0.010153,1,9,1,65
3,595,33.0,7,2,0.016932,1,9,1,65
4,145,38.0,7,1,0.014390,1,9,1,65


In [637]:
transactions_processed = pd.get_dummies(transactions_processed, columns=['sales_channel_id', 'colour_type'])
transactions_processed.head()

,customer_id,age,article_id,price,ordered,colour_group_code,count,sales_channel_id_1,sales_channel_id_2,colour_type_0,...,colour_type_728,colour_type_729,colour_type_730,colour_type_731,colour_type_732,colour_type_733,colour_type_734,colour_type_735,colour_type_736,colour_type_737
0,8,28.0,7,0.016932,1,9,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
1,8,28.0,7,0.016932,1,9,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
2,8,28.0,7,0.010153,1,9,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0
3,595,33.0,7,0.016932,1,9,1,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,145,38.0,7,0.014390,1,9,1,1,0,0,...,0,0,0,0,0,0,0,0,0,0


In [638]:
X_train, X_test, y_train, y_test = train_test_split(transactions_processed.drop('ordered', axis=1), transactions_processed['ordered'], test_size=0.10, random_state=42)
baseline = LogisticRegression(random_state=42)
baseline = baseline.fit(X_train, y_train)
baseline.score(X_test, y_test)

/usr/local/lib/python3.7/dist-packages/sklearn/linear_model/_logistic.py:818: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG,


0.6253229974160207